In [1]:
!pip install -q transformers torch datasets pandas scikit-learn accelerate bitsandbytes peft fuzzywuzzy python-levenshtein

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
import re
import logging
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
with open('data/categories.txt', 'r', encoding='utf-8') as f:
    categories = [line.strip() for line in f.readlines()]

print(f"Категории: {categories}")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Осмотр train
print("\nПримеры из train:")
print(train_df['text'].head(3).tolist())

# Проверка на NaN и дубликаты
print(f"NaN в train: {train_df.isnull().sum().sum()}")
train_df.drop_duplicates(subset=['text'], inplace=True)
print(f"Train после удаления дубликатов: {train_df.shape}")

Категории: ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']
Train shape: (1818, 1)
Test shape: (7276, 1)

Примеры из train:
['Заказали 14.10.2017 , получили 25.10.2017 \r\nНа мой размер 42, широкий как мешок. Надо было все таки размер  S заказать. \r\nПо поводу качества хороший пуховик. \r\nМех натуральный , съемный. \r\nБуду продавать .', 'футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой М, заказала Л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.', 'Все отлично!!!']
NaN в train: 0
Train после удаления дубликатов: (1818, 1)


In [4]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text.strip())
    text = re.sub(r'[^\w\s.,!?—–-]', ' ', text)
    if len(text) < 10:
        return ""
    return text

# Применение предобработки
train_df['text_clean'] = train_df['text'].apply(preprocess_text)
test_df['text_clean'] = test_df['text'].apply(preprocess_text)

# Удаление пустых после очистки
train_df = train_df[train_df['text_clean'] != ''].reset_index(drop=True)
test_df = test_df[test_df['text_clean'] != ''].reset_index(drop=True)

print(f"Train после предобработки: {train_df.shape}")
print("\nПримеры очищенных текстов:")
print(train_df['text_clean'].head(3).tolist())

Train после предобработки: (1803, 2)

Примеры очищенных текстов:
['заказали 14.10.2017 , получили 25.10.2017 на мой размер 42, широкий как мешок. надо было все таки размер s заказать. по поводу качества хороший пуховик. мех натуральный , съемный. буду продавать .', 'футболка хорошего качества,но футболка не как для девушек и женщин,а как на мужчину. она очень свободная. на свой м, заказала л. теперь не знаю что делать,ибо она мне велика, даже моему папе она полезет.', 'все отлично!!!']


In [5]:
# Модель для разметки (mGPT от Sber, открытая, русская)
model_name = "ai-forever/mGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 4-bit квантизация для T4 (экономит VRAM)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto" if device == "cuda" else None,
    trust_remote_code=False,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# Проверка загрузки
if device == "cuda":
    print(f"VRAM в использовании: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Модель загружена на {device}. Размер: {base_model.num_parameters() / 1e9:.1f}B параметров")

`torch_dtype` is deprecated! Use `dtype` instead!


VRAM в использовании: 1.06 GB
Модель загружена на cuda. Размер: 1.4B параметров


In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

few_shot_examples = """Ты классификатор отзывов по категориям товаров. Для каждого отзыва определи категорию из списка: бытовая техника, обувь, одежда, посуда, текстиль, товары для детей, украшения и аксессуары, электроника, нет товара. Следуй этим шагам:
1. Прочитай отзыв.
2. Найди ключевые слова, указывающие на товар (например, "футболка" → одежда, "холодильник" → бытовая техника).
3. Если упоминаний товара нет (например, только про доставку или спор), выбери "нет товара".
4. Выведи только название категории.

Примеры:
1. Отзыв: "Футболка классная, но маловата, берите на размер больше." Категория: одежда
2. Отзыв: "Пуховик тёплый, мех натуральный, но велик." Категория: одежда
3. Отзыв: "Джинсы пришли с пятнами, качество так себе." Категория: одежда
4. Отзыв: "Кроссовки жмут, быстро износились." Категория: обувь
5. Отзыв: "Туфли не подошли, но доставка быстрая." Категория: обувь
6. Отзыв: "Ботинки тёплые, но швы кривые." Категория: обувь
7. Отзыв: "Холодильник шумит, но охлаждает нормально." Категория: бытовая техника
8. Отзыв: "Микроволновка быстро сломалась, спор не помог." Категория: бытовая техника
9. Отзыв: "Пылесос мощный, но тяжёлый." Категория: бытовая техника
10. Отзыв: "Тарелки красивые, но одна разбилась при доставке." Категория: посуда
11. Отзыв: "Кружка с трещиной, качество плохое." Категория: посуда
12. Отзыв: "Чайник керамический, но ручка шатается." Категория: посуда
13. Отзыв: "Полотенце мягкое, но быстро износилось." Категория: текстиль
14. Отзыв: "Шторы тонкие, просвечивают, не советую." Категория: текстиль
15. Отзыв: "Простыня приятная, но размер не подошёл." Категория: текстиль
16. Отзыв: "Игрушка яркая, ребёнку нравится." Категория: товары для детей
17. Отзыв: "Коляска сломалась через месяц." Категория: товары для детей
18. Отзыв: "Погремушка безопасная, но быстро надоела." Категория: товары для детей
19. Отзыв: "Серьги потемнели, выглядят дёшево." Категория: украшения и аксессуары
20. Отзыв: "Браслет красивый, но застёжка слабая." Категория: украшения и аксессуары
21. Отзыв: "Часы не работают, подделка." Категория: украшения и аксессуары
22. Отзыв: "Телефон быстро разряжается, не советую." Категория: электроника
23. Отзыв: "Наушники с плохим звуком, быстро сломались." Категория: электроника
24. Отзыв: "Чехол для телефона тонкий, но стильный." Категория: электроника
25. Отзыв: "Заказ не пришёл, деньги не вернули." Категория: нет товара
26. Отзыв: "Доставка три месяца, спор не помог." Категория: нет товара
27. Отзыв: "Товар не отслеживался, продавец не отвечает." Категория: нет товара
"""

keyword_map = {
    "одежда": ["футболка", "пуховик", "блузка", "юбка", "штаны", "джинсы", "свитер", "куртка", "платье", "кофта", "шорты", "рубашка", "комбинезон", "халат", "кардиган", "майка", "туника", "водолазка", "жилет", "пижама", "толстовка"],
    "обувь": ["кроссовки", "туфли", "ботинки", "сапоги", "тапки", "сандалии", "кеды"],
    "бытовая техника": ["холодильник", "микроволновка", "пылесос", "стиральная машина", "чайник", "утюг", "блендер", "фен"],
    "посуда": ["тарелки", "кружка", "чашка", "ложка", "вилка", "нож", "кастрюля", "сковорода", "бокал", "чайник керамический"],
    "текстиль": ["полотенце", "шторы", "постельное", "одеяло", "подушка", "скатерть", "простыня", "наволочка"],
    "товары для детей": ["игрушка", "погремушка", "пеленка", "коляска", "кроватка", "памперс"],
    "украшения и аксессуары": ["серьги", "браслет", "кольцо", "цепочка", "кулон", "часы", "ремень", "шарф"],
    "электроника": ["телефон", "наушники", "планшет", "зарядка", "чехол", "кабель", "ноутбук", "клавиатура"],
    "нет товара": ["доставка", "спор", "возврат", "не пришёл", "не дошло", "деньги", "задержка", "отслеживание"]
}

def classify_review(text, model, tokenizer, categories, few_shot=few_shot_examples, device="cuda"):
    """
    Классифицирует отзыв по категории с использованием mGPT.
    """
    try:
        prompt = f"""{few_shot}
Отзыв: "{text}"
Категория:"""

        # Токенизация
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
            return_attention_mask=True
        ).to(device)

        # Генерация
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=10,
                temperature=0.05,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_return_sequences=1,
                use_cache=True
            )

        # Декодирование
        full_generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred = full_generated.split("Категория:")[-1].strip().lower()

        # Fuzzy matching
        best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))
        confidence = fuzz.ratio(pred, best_match.lower()) / 100.0

        # Страховка: ключевые слова
        text_lower = text.lower()
        keyword_category = None
        for category, keywords in keyword_map.items():
            if any(keyword in text_lower for keyword in keywords):
                keyword_category = category
                break

        # Логика: если модель уверена (confidence >= 0.7) или есть ключевое слово
        if confidence >= 0.7:
            logger.info(f"Модель: {best_match} (Уверенность: {confidence:.2f}) для '{text[:50]}...'")
            return best_match
        elif keyword_category:
            logger.info(f"Ключевое слово: {keyword_category} для '{text[:50]}...'")
            return keyword_category
        else:
            logger.info(f"Низкая уверенность ({confidence:.2f}), нет ключевых слов: 'нет товара' для '{text[:50]}...'")
            return "нет товара"

    except Exception as e:
        logger.error(f"Ошибка при классификации отзыва '{text[:50]}...': {str(e)}")
        return "нет товара"

test_text = train_df['text_clean'].iloc[0]
start_time = time.time()
pred = classify_review(test_text, base_model, tokenizer, categories)
elapsed = time.time() - start_time
print(f"Отзыв: {test_text[:100]}...")
print(f"Предсказание: {pred}")
print(f"Время классификации: {elapsed:.2f} сек")

Отзыв: заказали 14.10.2017 , получили 25.10.2017 на мой размер 42, широкий как мешок. надо было все таки ра...
Предсказание: товары для детей
Время классификации: 2.39 сек


In [7]:
batch_size = 50
train_labeled = []

start_time_total = time.time()
for i in range(0, len(train_df), batch_size):
    batch = train_df['text_clean'][i:i+batch_size]
    batch_preds = []

    batch_start = time.time()
    for text in batch:
        pred = classify_review(text, base_model, tokenizer, categories)
        batch_preds.append(pred)

    train_labeled.extend(batch_preds)
    elapsed_batch = time.time() - batch_start
    print(f"Батч {i//batch_size + 1}: {len(batch)} примеров за {elapsed_batch:.2f}с (ср. {elapsed_batch/len(batch):.2f}с/пример)")
    torch.cuda.empty_cache() if device == "cuda" else None

total_time = time.time() - start_time_total
print(f"Общее время разметки: {total_time:.2f}с (ср. {total_time/len(train_df):.2f}с/пример)")

train_df_labeled = train_df.copy()
train_df_labeled['label'] = train_labeled
train_df_labeled[['text_clean', 'label']].to_csv('train_labeled.csv', index=False)
print(f"Сохранено в train_labeled.csv: {train_df_labeled.shape}")
print("\nРаспределение классов:")
print(train_df_labeled['label'].value_counts())

Батч 1: 50 примеров за 51.72с (ср. 1.03с/пример)
Батч 2: 50 примеров за 40.81с (ср. 0.82с/пример)
Батч 3: 50 примеров за 36.76с (ср. 0.74с/пример)
Батч 4: 50 примеров за 39.45с (ср. 0.79с/пример)
Батч 5: 50 примеров за 21.73с (ср. 0.43с/пример)
Батч 6: 50 примеров за 21.33с (ср. 0.43с/пример)
Батч 7: 50 примеров за 21.65с (ср. 0.43с/пример)
Батч 8: 50 примеров за 21.73с (ср. 0.43с/пример)
Батч 9: 50 примеров за 21.79с (ср. 0.44с/пример)
Батч 10: 50 примеров за 21.69с (ср. 0.43с/пример)
Батч 11: 50 примеров за 21.63с (ср. 0.43с/пример)
Батч 12: 50 примеров за 21.58с (ср. 0.43с/пример)
Батч 13: 50 примеров за 21.68с (ср. 0.43с/пример)
Батч 14: 50 примеров за 21.64с (ср. 0.43с/пример)
Батч 15: 50 примеров за 21.65с (ср. 0.43с/пример)
Батч 16: 50 примеров за 21.14с (ср. 0.42с/пример)
Батч 17: 50 примеров за 21.55с (ср. 0.43с/пример)
Батч 18: 50 примеров за 21.57с (ср. 0.43с/пример)
Батч 19: 50 примеров за 22.09с (ср. 0.44с/пример)
Батч 20: 50 примеров за 21.40с (ср. 0.43с/пример)
Батч 21: 

In [8]:
print("Распределение по категориям:")
label_counts = train_df_labeled['label'].value_counts()
print(label_counts)


for cat in categories[:3]:
    examples = train_df_labeled[train_df_labeled['label'] == cat]['text_clean'].head(1).tolist()
    if examples:
        print(f"\n{cat}: {examples[0][:100]}...")

print("\nГотово! Теперь можно перейти к fine-tune в отдельном notebook'e, используя train_labeled.csv")

Распределение по категориям:
label
нет товара                1065
товары для детей           370
одежда                     304
бытовая техника             54
украшения и аксессуары       5
посуда                       4
текстиль                     1
Name: count, dtype: int64

бытовая техника: плохо прошиты швы, в этих местах материал собирается...

одежда: заказали 14.10.2017 , получили 25.10.2017 на мой размер 42, широкий как мешок. надо было все таки ра...

Готово! Теперь можно перейти к fine-tune в отдельном notebook'e, используя train_labeled.csv
